## SecSSE : Reproduction 

In [1]:
## SecSSE : Reproduction 

## I Data preparation

In [2]:
## I Data preparation

### Loading packages

In [3]:
### Loading packages

In [4]:
library("secsse")
library("stringr")
library("qpcR")

Le chargement a nécessité le package : ape

Le chargement a nécessité le package : MASS

Le chargement a nécessité le package : minpack.lm

Le chargement a nécessité le package : rgl

This build of rgl does not include OpenGL functions.  Use
 rglwidget() to display results, e.g. via options(rgl.printRglwidget = TRUE).

Le chargement a nécessité le package : robustbase

Le chargement a nécessité le package : Matrix



### Loading data

In [5]:
### Loading data

In [6]:
df<-read.csv("Path/to/your/file.tsv", sep="\t") # omit sep ="\t" for .csv files
phy<-read.nexus("Path/to/your/tree.tree")

In [7]:
states<-df$Reproduction
names(states)=str_replace((rownames(df)), " ", "_")
states2<-states[!names(states) %in% setdiff(names(states), phy$tip.label)]

In [8]:
phy2.0<-drop.tip(phy, setdiff(phy$tip.label, (names(states))))

### Computing sampling fraction

In [9]:
### Computing sampling fraction

In [10]:
samp_frac<-c(
length(states2[states2=="YV"])/length(df$Reproduction[df$Reproduction=="YV"]),
length(states2[states2=="O"])/length(df$Reproduction[df$Reproduction=="O"]),
length(states2[states2=="Oo"])/length(df$Reproduction[df$Reproduction=="Oo"]),
length(states2[states2=="PV"])/length(df$Reproduction[df$Reproduction=="PV"]))

### Naming states according to Secsse

In [11]:
### Naming states according to Secsse

In [12]:
states3<-as.data.frame(cbind(names(states2), as.factor(states2)))
colnames(states3)<-c("species","states")
rownames(states3)<-NULL
traits <- sortingtraits(states3, phy2.0)

## II Running the analysis

In [13]:
## II Running the analysis

### Initialisation

In [14]:
### Initialisation

In [15]:
startingpoint <- bd_ML(brts = ape::branching.times(phy2.0))
intGuessLambda <- startingpoint$lambda0
intGuessMu <- startingpoint$mu0
#COND=conditioning on the state of the root
cond<-"proper_cond"
#root_state_weight
root_state_weight<-"proper_weights"
#SAMPLING_FRACTION
#TOL
tol = c(1e-04, 1e-05, 1e-07)

You are optimizing lambda0 mu0 
You are fixing lambda1 mu1 
Optimizing the likelihood - this may take a while. 
The loglikelihood for the initial parameter values is -2042.37935685298.

Maximum likelihood parameter estimates: lambda0: 0.028186, mu0: 0.009724, lambda1: 0.000000, mu1: 0.000000: 
Maximum loglikelihood: -1780.702572


### Constant rate model

In [16]:
### Constant rate model

In [17]:
  idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][] <- 1 #same speciation rate
  idparslist[[2]][] <- 2 #same extinction rate
  
  #same transition rate
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 3
  masterBlock[1,3] <- 3
  masterBlock[1,4] <- 3
  masterBlock[2,1] <- 3
  masterBlock[2,3] <- 3
  masterBlock[2,4] <- 3
  masterBlock[3,1] <- 3
  masterBlock[3,2] <- 3
  masterBlock[3,4] <- 3
  masterBlock[4,1] <- 3
  masterBlock[4,2] <- 3
  masterBlock[4,3] <- 3
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 4, 3)
  idparslist[[3]] <- myQ
  myQ

NA,3,3,3,3,0,0,0,3,0,0,0,3,0,0,0
3,NA,3,3,0,3,0,0,0,3,0,0,0,3,0,0
3,3,NA,3,0,0,3,0,0,0,3,0,0,0,3,0
3,3,3,NA,0,0,0,3,0,0,0,3,0,0,0,3
3,0,0,0,NA,3,3,3,3,0,0,0,3,0,0,0
0,3,0,0,3,NA,3,3,0,3,0,0,0,3,0,0
0,0,3,0,3,3,NA,3,0,0,3,0,0,0,3,0
0,0,0,3,3,3,3,NA,0,0,0,3,0,0,0,3
3,0,0,0,3,0,0,0,NA,3,3,3,3,0,0,0
0,3,0,0,0,3,0,0,3,NA,3,3,0,3,0,0
0,0,3,0,0,0,3,0,3,3,NA,3,0,0,3,0


In [18]:
#INITPARSOPT = initiale values of each parameters
initparsopt <- c(intGuessLambda, intGuessMu, 0.25) # speciation rate, extinction rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:3)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CR_sp<-secsse_ml(phy2.0, traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2287.346 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0097240310612135 0.25 -2287.34637051797 initial 
2 0.0295954409554525 0.0097240310612135 0.25 -2285.5793214581 reflect 
3 0.0305371270580805 0.00923829728847183 0.23371335504886 -2284.82345897371 expand 
4 0.0305371270580805 0.00923829728847183 0.23371335504886 -2284.82345897371 reflect 
5 0.0318478857083907 0.00945412125781383 0.227493698235506 -2284.80750997466 reflect 
6 0.0318478857083907 0.00945412125781383 0.227493698235506 -2284.80750997466 contract inside 
7 0.030153269305312 0.010030103379914 0.215096245247148 -2284.3561072966 expand 
8 0.031707912610423 0.0097420301802166 0.192426539179104 -2283.44826957726 expand 
9 0.0326359426112582 0.0107510060156801 0.169382504288164 -2282.15778111517 expand 
10 0.0307990123109034 0.01161744872

88 0.0331079674418965 0.0138578868427002 0.000409236064402192 -1874.18550152378 contract inside 
89 0.0331079674418965 0.0138578868427002 0.000409236064402192 -1874.18550152378 contract inside 
90 0.0331081469088884 0.0138579494593698 0.000409324170526836 -1874.18549705907 reflect 
91 0.0331035760343965 0.0138571966744493 0.000409726002781586 -1874.18549130898 contract outside 
92 0.0330976649277599 0.0138563167600284 0.000409476910040828 -1874.18548449875 contract inside 
93 0.0330976649277599 0.0138563167600284 0.000409476910040828 -1874.18548449875 reflect 
94 0.0330976649277599 0.0138563167600284 0.000409476910040828 -1874.18548449875 contract inside 
95 0.0330963916609001 0.0138561605083028 0.00040944186646737 -1874.18548291351 reflect 
96 0.0330963916609001 0.0138561605083028 0.00040944186646737 -1874.18548291351 contract outside 
97 0.0331009872199529 0.0138568409059716 0.000409452054697661 -1874.18548247414 contract inside 
98 0.0331005079434654 0.0138568017743532 0.00040932893

178 0.0341708877052437 0.0156158014049137 0.000409650605218717 -1874.11845287184 reflect 
179 0.0341513253115651 0.0155883211276504 0.000409538161145478 -1874.11845035313 reflect 
180 0.0341513253115651 0.0155883211276504 0.000409538161145478 -1874.11845035313 reflect 
181 0.0341610284045254 0.0156013326031153 0.000409219961373255 -1874.1184500763 contract inside 
182 0.0341603232475444 0.0156062989438309 0.000409400600344111 -1874.11844769582 contract inside 
183 0.0341642232941805 0.0156072260475273 0.000409518423060549 -1874.11844682005 contract inside 
184 0.0341565917852788 0.0155966367581857 0.00040945891135525 -1874.11844661297 contract inside 
185 0.0341565917852788 0.0155966367581857 0.00040945891135525 -1874.11844661297 contract outside 
186 0.0341603066235527 0.0156045290169857 0.000409459687025466 -1874.11844628208 contract inside 
187 0.0341605333637658 0.0156019886122295 0.000409429017360558 -1874.11844614437 contract outside 
188 0.0341616836013742 0.0156041387402851 0.0

### CTD2 model (Two conceal states)

In [19]:
### CTD2 model (Two conceal states)

In [20]:
  idparslist <- id_paramPos(traits, num_concealed_states = 2)
  idparslist[[1]][1:4] <- 1 # Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[2]][] <- 3 # same extinction rate

  diag(idparslist[[3]]) <- NA # remove same state transition rate (diagonal)
  # same transition rates
  idparslist[[3]][1,c(2,3,4,5)] <- 4
  idparslist[[3]][1,c(6,7,8)] <- 0
  idparslist[[3]][2,c(1,3,4,6)] <- 4
  idparslist[[3]][2,c(5,7,8)] <- 0
  idparslist[[3]][3,c(1,2,4,7)] <- 4
  idparslist[[3]][3,c(5,6,8)] <- 0
  idparslist[[3]][4,c(1,2,3,8)] <- 4
  idparslist[[3]][4,c(5,6,7)] <- 0
  idparslist[[3]][5,c(1,6,7,8)] <- 4
  idparslist[[3]][5,c(2,3,4)] <- 0
  idparslist[[3]][6,c(2,5,7,8)] <- 4
  idparslist[[3]][6,c(1,3,4)] <- 0
  idparslist[[3]][7,c(3,5,6,8)] <- 4
  idparslist[[3]][7,c(1,2,4)] <- 0
  idparslist[[3]][8,c(4,5,6,7)] <- 4
  idparslist[[3]][8,c(1,2,3,8)] <- 0
  idparslist[[3]]

,1A,2A,3A,4A,1B,2B,3B,4B
1A,NA,4,4,4,4,0,0,0
2A,4,NA,4,4,0,4,0,0
3A,4,4,NA,4,0,0,4,0
4A,4,4,4,NA,0,0,0,4
1B,4,0,0,0,NA,4,4,4
2B,0,4,0,0,4,NA,4,4
3B,0,0,4,0,4,4,NA,4
4B,0,0,0,4,4,4,4,0


In [21]:
#INITPARSOPT = initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,2), intGuessMu, 0.25)# speciation rate, extinction rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:4)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD2_sp<-secsse_ml(phy2.0,traits, num_concealed_states=2, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2287.346 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2287.34637051804 initial 
2 0.0292427522080496 0.0292427522080496 0.00875303062128823 0.259351620947631 -2285.96292115982 expand 
3 0.0300366419406499 0.0300366419406499 0.00899560560880712 0.232458816351434 -2284.97071991618 expand 
4 0.0300366419406499 0.0300366419406499 0.00899560560880712 0.232458816351434 -2284.97071991618 reflect 
5 0.0300366419406499 0.0300366419406499 0.00899560560880712 0.232458816351434 -2284.97071991618 reflect 
6 0.0300366419406499 0.0300366419406499 0.00899560560880712 0.232458816351434 -2284.97071991618 reflect 
7 0.0300366419406499 0.0300366419406499 0.00899560560880712 0.232458816351434 -2284.97071991618 reflect 
8 0.0310390201263572 0.0291849122820341 0.00950387502734

74 0.0267303704012777 0.0409043545171399 0.0165399201440658 0.000473416481778842 -1871.79502551151 reflect 
75 0.0267303704012777 0.0409043545171399 0.0165399201440658 0.000473416481778842 -1871.79502551151 reflect 
76 0.0268373259831507 0.040929700608301 0.0162930246978853 0.000470153085183319 -1871.75144268709 expand 
77 0.0268373259831507 0.040929700608301 0.0162930246978853 0.000470153085183319 -1871.75144268709 reflect 
78 0.0268373259831507 0.040929700608301 0.0162930246978853 0.000470153085183319 -1871.75144268709 reflect 
79 0.0268373259831507 0.040929700608301 0.0162930246978853 0.000470153085183319 -1871.75144268709 reflect 
80 0.0268373259831507 0.040929700608301 0.0162930246978853 0.000470153085183319 -1871.75144268709 reflect 
81 0.0270285182278177 0.0409873665808275 0.0158152945990403 0.000479061840654102 -1871.70645960103 expand 
82 0.0270285182278177 0.0409873665808275 0.0158152945990403 0.000479061840654102 -1871.70645960103 reflect 
83 0.0270285182278177 0.04098736658

149 0.0236628089955929 0.0478205739166183 0.00348684906178192 0.000423533847750085 -1868.02918345865 reflect 
150 0.0223660463276964 0.0495665624866113 0.00115071214293202 0.000402049000963961 -1867.7818723265 expand 
151 0.0223167446941279 0.0494739956276685 0.00146840245694713 0.000396215337756182 -1867.76105734143 reflect 
152 0.0223167446941279 0.0494739956276685 0.00146840245694713 0.000396215337756182 -1867.76105734143 reflect 
153 0.0223167446941279 0.0494739956276685 0.00146840245694713 0.000396215337756182 -1867.76105734143 reflect 
154 0.0223167446941279 0.0494739956276685 0.00146840245694713 0.000396215337756182 -1867.76105734143 reflect 
155 0.0220765727676099 0.049873202286832 0.000844643286929192 0.000396585801376324 -1867.7500795877 contract outside 
156 0.0218293883646114 0.0499321225628738 0.00110519514507944 0.000387521045143746 -1867.72112015341 contract inside 
157 0.0218293883646114 0.0499321225628738 0.00110519514507944 0.000387521045143746 -1867.72112015341 refle

222 0.020455038314354 0.0512569059291491 1.58103935825221e-06 0.000491735985364516 -1867.05212738139 contract inside 
223 0.020455038314354 0.0512569059291491 1.58103935825221e-06 0.000491735985364516 -1867.05212738139 contract inside 
224 0.020455038314354 0.0512569059291491 1.58103935825221e-06 0.000491735985364516 -1867.05212738139 contract inside 
225 0.020455038314354 0.0512569059291491 1.58103935825221e-06 0.000491735985364516 -1867.05212738139 contract inside 
226 0.020455038314354 0.0512569059291491 1.58103935825221e-06 0.000491735985364516 -1867.05212738139 contract inside 
227 0.0204576644413541 0.0512547484194201 2.75698451389889e-06 0.000491590125923118 -1867.05211886541 reflect 
228 0.0204576644413541 0.0512547484194201 2.75698451389889e-06 0.000491590125923118 -1867.05211886541 contract inside 
229 0.0204576644413541 0.0512547484194201 2.75698451389889e-06 0.000491590125923118 -1867.05211886541 reflect 
230 0.0204544022443953 0.0512578245751761 2.67841843940662e-07 0.0004

294 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 contract inside 
295 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 reflect 
296 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 reflect 
297 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 contract inside 
298 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 contract inside 
299 0.0204599268174965 0.0512543719614863 3.54334181702655e-09 0.000490858057865679 -1867.05208174977 reflect 
300 0.0204602522623937 0.0512541598651733 7.28527975054098e-10 0.00049085163771184 -1867.05208174625 expand 
301 0.0204602522623937 0.0512541598651733 7.28527975054098e-10 0.00049085163771184 -1867.05208174625 contract inside 
302 0.0204602522623937 0.0512541598651733 7.28527975054098e-10 0.00049085163771184 

365 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 reflect 
366 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 contract inside 
367 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 contract inside 
368 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 reflect 
369 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 contract inside 
370 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 contract inside 
371 0.0204600653679711 0.0512542813795876 3.33617172531188e-13 0.000490870264980444 -1867.0520817212 contract inside 
372 0.0204600644971591 0.0512542819432397 4.0449879243033e-13 0.000490870366942153 -1867.0520817212 reflect 
373 0.0204600644971591 0.0512542819432397 4.0449879243033e-13 0.00049087036694215

### CTD3 model (Three conceal states)

In [22]:
### CTD3 model (Three conceal states)

In [23]:
  idparslist <- id_paramPos(traits, num_concealed_states = 3)
  idparslist[[1]][1:4] <- 1 #Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[1]][9:12] <- 3
  idparslist[[2]][] <- 4 #same extinction rate
  
  diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
  #all transition rates equal
  idparslist[[3]][1,c(2,3,4,5,9)] <- 5
  idparslist[[3]][1,c(6,7,8,10,11,12)] <- 0
  idparslist[[3]][2,c(1,3,4,6,10)] <- 5
  idparslist[[3]][2,c(5,7,8,9,11,12)] <- 0
  idparslist[[3]][3,c(1,2,4,7,11)] <- 5
  idparslist[[3]][3,c(5,6,8,9,10,12)] <- 0
  idparslist[[3]][4,c(1,2,3,8,12)] <- 5
  idparslist[[3]][4,c(5,6,7,9,10,11)] <- 0
  idparslist[[3]][5,c(1,6,7,8,9)] <- 5
  idparslist[[3]][5,c(2,3,4,10,11,12)] <- 0
  idparslist[[3]][6,c(2,5,7,8,10)] <- 5
  idparslist[[3]][6,c(1,3,4,9,11,12)] <- 0
  idparslist[[3]][7,c(3,5,6,8,11)] <- 5
  idparslist[[3]][7,c(1,2,4,9,10,12)] <- 0
  idparslist[[3]][8,c(4,5,6,7,12)] <- 5
  idparslist[[3]][8,c(1,2,3,9,10,11)] <- 0
  idparslist[[3]][9,c(1,5,10,11,12)] <- 5
  idparslist[[3]][9,c(2,3,4,6,7,8)] <- 0
  idparslist[[3]][10,c(2,6,9,11,12)] <- 5
  idparslist[[3]][10,c(1,3,4,5,7,8)] <- 0
  idparslist[[3]][11,c(3,7,9,10,12)] <- 5
  idparslist[[3]][11,c(1,2,4,5,6,8)] <- 0
  idparslist[[3]][12,c(4,8,9,10,11)] <- 5
  idparslist[[3]][12,c(1,2,3,5,6,7)] <- 0
  idparslist[[3]]

,1A,2A,3A,4A,1B,2B,3B,4B,1C,2C,3C,4C
1A,NA,5,5,5,5,0,0,0,5,0,0,0
2A,5,NA,5,5,0,5,0,0,0,5,0,0
3A,5,5,NA,5,0,0,5,0,0,0,5,0
4A,5,5,5,NA,0,0,0,5,0,0,0,5
1B,5,0,0,0,NA,5,5,5,5,0,0,0
2B,0,5,0,0,5,NA,5,5,0,5,0,0
3B,0,0,5,0,5,5,NA,5,0,0,5,0
4B,0,0,0,5,5,5,5,NA,0,0,0,5
1C,5,0,0,0,5,0,0,0,NA,5,5,5
2C,0,5,0,0,0,5,0,0,5,NA,5,5


In [24]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,3), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:5)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD3_sp<-secsse_ml(phy2.0,traits, num_concealed_states=3, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2287.346 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2287.3463705177 initial 
2 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.00875303062128823 0.257470119521912 -2286.09460339722 expand 
3 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2285.1469134137 expand 
4 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2285.1469134137 reflect 
5 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2285.1469134137 reflect 
6 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2285.1469134137 reflect 
7 0.0295389945138178 0.0295389945138178 0.0295389945

63 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
64 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
65 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
66 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
67 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
68 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
69 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0150880596811424 0.000430315304259765 -1871.96951967159 contract inside 
70 0.031761646596773 0.0336597510429497 0.0455253918997353 0.0

127 0.024217120927266 0.0429529188638557 0.0791586080819505 0.00577191482501888 0.000442881153132814 -1864.84143206005 reflect 
128 0.024217120927266 0.0429529188638557 0.0791586080819505 0.00577191482501888 0.000442881153132814 -1864.84143206005 contract inside 
129 0.024217120927266 0.0429529188638557 0.0791586080819505 0.00577191482501888 0.000442881153132814 -1864.84143206005 contract inside 
130 0.024217120927266 0.0429529188638557 0.0791586080819505 0.00577191482501888 0.000442881153132814 -1864.84143206005 reflect 
131 0.024217120927266 0.0429529188638557 0.0791586080819505 0.00577191482501888 0.000442881153132814 -1864.84143206005 reflect 
132 0.0234836299493134 0.0422887163790745 0.082187745840955 0.0038669889778731 0.000440213327116028 -1864.83034760708 contract inside 
133 0.0242292927060161 0.0420579822129212 0.0788927062372556 0.00522721047439496 0.000441078239840181 -1864.82098283859 contract inside 
134 0.0242292927060161 0.0420579822129212 0.0788927062372556 0.005227210

189 0.0238197536144636 0.0426761587316839 0.080819103655869 0.00479295667226732 0.00041412877254263 -1864.75243362811 expand 
190 0.0238197536144636 0.0426761587316839 0.080819103655869 0.00479295667226732 0.00041412877254263 -1864.75243362811 reflect 
191 0.0238197536144636 0.0426761587316839 0.080819103655869 0.00479295667226732 0.00041412877254263 -1864.75243362811 reflect 
192 0.0238096037646069 0.0425843138616888 0.0808318401537161 0.0047206562298026 0.000412826670146857 -1864.75227969946 expand 
193 0.0238096037646069 0.0425843138616888 0.0808318401537161 0.0047206562298026 0.000412826670146857 -1864.75227969946 reflect 
194 0.0238201093962067 0.042568977378215 0.0807794004386752 0.00473422296027032 0.000413398996765365 -1864.75180881876 expand 
195 0.0237650423587534 0.0426335617154327 0.0810295808813338 0.00466716431157701 0.000413030675477308 -1864.75155382898 expand 
196 0.0237928672289764 0.0425812498567541 0.0808894433790658 0.00469477618837257 0.00041234044014037 -1864.750

254 0.0201676621415587 0.034825690054155 0.0809430275269622 0.00187079299263823 0.000496098121768568 -1863.89866639705 reflect 
255 0.0201676621415587 0.034825690054155 0.0809430275269622 0.00187079299263823 0.000496098121768568 -1863.89866639705 reflect 
256 0.0202498597378012 0.0351565154865449 0.0796224395264359 0.00296048911003057 0.000473249013830382 -1863.86578226407 expand 
257 0.0202498597378012 0.0351565154865449 0.0796224395264359 0.00296048911003057 0.000473249013830382 -1863.86578226407 reflect 
258 0.0202533514755363 0.0348939891637786 0.0798622140909835 0.00258118259729315 0.000479951645410245 -1863.84464331103 reflect 
259 0.0191107643861999 0.0334347813444356 0.0810629264524483 0.00167218208410115 0.000493939280544247 -1863.84105264062 reflect 
260 0.0204322073017139 0.0346500330867671 0.0793058861147061 0.00259494610986488 0.000463508192959107 -1863.81992799803 reflect 
261 0.0204322073017139 0.0346500330867671 0.0793058861147061 0.00259494610986488 0.00046350819295910

316 0.0195775870664056 0.0337517396194972 0.0807315186027141 0.0017118257606312 0.000462996116065234 -1863.78894412781 contract inside 
317 0.0195775870664056 0.0337517396194972 0.0807315186027141 0.0017118257606312 0.000462996116065234 -1863.78894412781 reflect 
318 0.0195801784403223 0.0337551682991956 0.0807303931660966 0.00171289116882161 0.000463035077753546 -1863.78894411063 contract inside 
319 0.0195779515988832 0.0337521873025716 0.0807351407247305 0.00170937974807042 0.000463001694580934 -1863.78894399106 reflect 
320 0.0195779515988832 0.0337521873025716 0.0807351407247305 0.00170937974807042 0.000463001694580934 -1863.78894399106 reflect 
321 0.0195788536539362 0.0337540373912943 0.0807332354517034 0.00171130650127306 0.00046304158167454 -1863.78894397073 contract inside 
322 0.0195801354372765 0.0337545860092931 0.080730861416146 0.00171214835211111 0.000462983296017524 -1863.78894395992 contract inside 
323 0.0195785508072445 0.0337532702663016 0.0807320620711654 0.001711

### CTD4 model (Four conceal states)

In [25]:
### CTD4 model (Four conceal states)

In [26]:
  idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][1:4] <- 1 #Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[1]][9:12] <- 3  
  idparslist[[1]][13:16] <- 4 
  idparslist[[2]][] <- 5 #same extinction rate
  
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 6
  masterBlock[1,3] <- 6
  masterBlock[1,4] <- 6
  masterBlock[2,1] <- 6
  masterBlock[2,3] <- 6
  masterBlock[2,4] <- 6
  masterBlock[3,1] <- 6
  masterBlock[3,2] <- 6
  masterBlock[3,4] <- 6
  masterBlock[4,1] <- 6
  masterBlock[4,2] <- 6
  masterBlock[4,3] <- 6
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 7, 6)
  idparslist[[3]] <- myQ
  myQ

NA,6,6,6,6,0,0,0,6,0,0,0,6,0,0,0
6,NA,6,6,0,6,0,0,0,6,0,0,0,6,0,0
6,6,NA,6,0,0,6,0,0,0,6,0,0,0,6,0
6,6,6,NA,0,0,0,6,0,0,0,6,0,0,0,6
6,0,0,0,NA,6,6,6,6,0,0,0,6,0,0,0
0,6,0,0,6,NA,6,6,0,6,0,0,0,6,0,0
0,0,6,0,6,6,NA,6,0,0,6,0,0,0,6,0
0,0,0,6,6,6,6,NA,0,0,0,6,0,0,0,6
6,0,0,0,6,0,0,0,NA,6,6,6,6,0,0,0
0,6,0,0,0,6,0,0,6,NA,6,6,0,6,0,0
0,0,6,0,0,0,6,0,6,6,NA,6,0,0,6,0


In [27]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,4), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:6)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD4_sp<-secsse_ml(phy2.0,traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2287.346 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2287.34637051797 initial 
2 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00875303062128823 0.256218905472637 -2286.19680347946 expand 
3 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 expand 
4 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 reflect 
5 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 reflect 
6 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.029242752208049

56 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
57 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
58 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
59 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
60 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
61 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1875.75374205429 contract inside 
62 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.03385661829953

111 0.0442078988413266 0.0340911418575365 0.0316215051800861 0.0294349226826888 0.015210882437952 0.000459986809487549 -1871.5762791221 contract inside 
112 0.0439926822169978 0.0343460176481248 0.0316924564490928 0.0295932188638625 0.0150394944325821 0.000448608610633775 -1871.57479532535 contract inside 
113 0.0439926822169978 0.0343460176481248 0.0316924564490928 0.0295932188638625 0.0150394944325821 0.000448608610633775 -1871.57479532535 contract outside 
114 0.0439926822169978 0.0343460176481248 0.0316924564490928 0.0295932188638625 0.0150394944325821 0.000448608610633775 -1871.57479532535 reflect 
115 0.0440585648147732 0.0342798556390735 0.0316664826151767 0.0295638652989081 0.015079726084751 0.000457938752645913 -1871.57446777302 contract inside 
116 0.0441163021926428 0.0341630892424415 0.0316633634940126 0.0294457625934484 0.015173820312982 0.000454713840875432 -1871.57418575692 reflect 
117 0.0441163021926428 0.0341630892424415 0.0316633634940126 0.0294457625934484 0.0151738

167 0.051299597181203 0.0316308441885006 0.0325011952823375 0.0272512329231062 0.0105816398218002 0.000535649774571994 -1868.67168536714 reflect 
168 0.051299597181203 0.0316308441885006 0.0325011952823375 0.0272512329231062 0.0105816398218002 0.000535649774571994 -1868.67168536714 reflect 
169 0.051299597181203 0.0316308441885006 0.0325011952823375 0.0272512329231062 0.0105816398218002 0.000535649774571994 -1868.67168536714 reflect 
170 0.0542737399309275 0.0313529758165598 0.033130144220552 0.026896628878747 0.00751315291560021 0.000534193931461435 -1868.28237988608 expand 
171 0.0542737399309275 0.0313529758165598 0.033130144220552 0.026896628878747 0.00751315291560021 0.000534193931461435 -1868.28237988608 reflect 
172 0.0580320998507973 0.0295704483453398 0.0333614234673676 0.0254462213412656 0.00595824189735289 0.000600632364944723 -1867.82637010539 expand 
173 0.0580320998507973 0.0295704483453398 0.0333614234673676 0.0254462213412656 0.00595824189735289 0.000600632364944723 -18

222 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 contract inside 
223 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 contract inside 
224 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 reflect 
225 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 reflect 
226 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 contract inside 
227 0.0731538120840708 0.0220247270624359 0.0349519886006892 0.0183138406490555 3.30710039259511e-05 0.000457018100281574 -1862.78654707055 reflect 
228 0.0728954841763226 0.0221400319376622 0.0349717687242054 0.0183641015063926 0.

...

771 0.0655062447727383 0.0255497688124197 0.0381345858828378 0.0168136975793542 9.23047753684611e-09 0.000477935709877555 -1862.06105772191 reflect 
772 0.0654877913829806 0.0255548675819256 0.0381477156033418 0.0167994283104542 9.88636852656616e-09 0.000478180196077756 -1862.06105008512 reflect 
773 0.0655025920898423 0.025546470142658 0.0381445785384141 0.0167962679562348 1.19964349216111e-08 0.00047824282926848 -1862.0610201613 expand 
774 0.0655025920898423 0.025546470142658 0.0381445785384141 0.0167962679562348 1.19964349216111e-08 0.00047824282926848 -1862.0610201613 reflect 
775 0.0655025920898423 0.025546470142658 0.0381445785384141 0.0167962679562348 1.19964349216111e-08 0.00047824282926848 -1862.0610201613 reflect 
776 0.0655025920898423 0.025546470142658 0.0381445785384141 0.0167962679562348 1.19964349216111e-08 0.00047824282926848 -1862.0610201613 reflect 
777 0.0655025920898423 0.025546470142658 0.0381445785384141 0.0167962679562348 1.19964349216111e-08 0.00047824282926848

827 0.0652737503468249 0.025835645403334 0.0384198437597537 0.016344373510693 1.33058223354883e-06 0.000482648789825608 -1862.02907879155 reflect 
828 0.065036366133973 0.0260341708276986 0.0385265221056858 0.0162665526981254 1.73654978362729e-06 0.000481930040562287 -1862.02310550574 expand 
829 0.065036366133973 0.0260341708276986 0.0385265221056858 0.0162665526981254 1.73654978362729e-06 0.000481930040562287 -1862.02310550574 reflect 
830 0.0651316505386001 0.0260757645914391 0.0385303666588029 0.0161786763790188 2.23731983764856e-06 0.000482651121371083 -1862.01171078506 expand 
831 0.0651316505386001 0.0260757645914391 0.0385303666588029 0.0161786763790188 2.23731983764856e-06 0.000482651121371083 -1862.01171078506 reflect 
832 0.0651316505386001 0.0260757645914391 0.0385303666588029 0.0161786763790188 2.23731983764856e-06 0.000482651121371083 -1862.01171078506 reflect 
833 0.0651316505386001 0.0260757645914391 0.0385303666588029 0.0161786763790188 2.23731983764856e-06 0.000482651

883 0.0715007011617308 0.0322203087602268 0.0353418257669119 0.0168986637891924 4.01043902343546e-05 0.000454676118594527 -1861.27095922287 contract inside 
884 0.0715007011617308 0.0322203087602268 0.0353418257669119 0.0168986637891924 4.01043902343546e-05 0.000454676118594527 -1861.27095922287 reflect 
885 0.0729906588573729 0.0329805979873841 0.034868696157603 0.0166344504138096 4.67579252280349e-05 0.000455928026669449 -1861.26832241701 reflect 
886 0.0729906588573729 0.0329805979873841 0.034868696157603 0.0166344504138096 4.67579252280349e-05 0.000455928026669449 -1861.26832241701 contract outside 
887 0.0729906588573729 0.0329805979873841 0.034868696157603 0.0166344504138096 4.67579252280349e-05 0.000455928026669449 -1861.26832241701 reflect 
888 0.0729906588573729 0.0329805979873841 0.034868696157603 0.0166344504138096 4.67579252280349e-05 0.000455928026669449 -1861.26832241701 reflect 
889 0.0729906588573729 0.0329805979873841 0.034868696157603 0.0166344504138096 4.675792522803

937 0.0723276308207966 0.0328881143692789 0.0350118395837429 0.0168461109168164 4.46675657330645e-05 0.000465662848074167 -1861.25334529119 contract inside 
938 0.0723276308207966 0.0328881143692789 0.0350118395837429 0.0168461109168164 4.46675657330645e-05 0.000465662848074167 -1861.25334529119 contract outside 
939 0.0723135902854666 0.0328927343191123 0.0350160507228561 0.0168469765264403 4.46548932241753e-05 0.000465647267480189 -1861.2533446205 contract inside 
940 0.072308139470381 0.032884464037028 0.0350188904256333 0.0168466007240456 4.46102181995799e-05 0.000465770627123853 -1861.25334441626 contract inside 
941 0.072308139470381 0.032884464037028 0.0350188904256333 0.0168466007240456 4.46102181995799e-05 0.000465770627123853 -1861.25334441626 contract inside 
942 0.072308139470381 0.032884464037028 0.0350188904256333 0.0168466007240456 4.46102181995799e-05 0.000465770627123853 -1861.25334441626 contract outside 
943 0.072308139470381 0.032884464037028 0.0350188904256333 0.01

991 0.0723135447144458 0.0328865708318649 0.0350168582598792 0.016846431253689 4.46305952010407e-05 0.000465712326261209 -1861.25334401592 contract inside 
992 0.0723135447144458 0.0328865708318649 0.0350168582598792 0.016846431253689 4.46305952010407e-05 0.000465712326261209 -1861.25334401592 contract inside 
993 0.0723135447144458 0.0328865708318649 0.0350168582598792 0.016846431253689 4.46305952010407e-05 0.000465712326261209 -1861.25334401592 contract inside 
994 0.0723134967999379 0.0328864727400281 0.0350168904027925 0.0168464177217941 4.46301134716371e-05 0.000465712305320839 -1861.25334401592 contract inside 
995 0.0723134967999379 0.0328864727400281 0.0350168904027925 0.0168464177217941 4.46301134716371e-05 0.000465712305320839 -1861.25334401592 reflect 
996 0.0723135113278816 0.0328865150410032 0.0350168832383852 0.0168464141693364 4.46303210140983e-05 0.000465712192756767 -1861.2533440159 contract inside 
Optimization has terminated successfully. 


### ETD model (Focal trait)

In [28]:
### ETD model (Focal trait)

In [29]:
 idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][c(1,5,9,13)] <- 1 #Speciation varies only between the 4 states
  idparslist[[1]][c(2,6,10,14)] <- 2 
  idparslist[[1]][c(3,7,11,15)] <- 3
  idparslist[[1]][c(4,8,12,16)] <- 4
  idparslist[[2]][] <- 5 #same extinction rate
  
  #all transition rates equal
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 6
  masterBlock[1,3] <- 6
  masterBlock[1,4] <- 6
  masterBlock[2,1] <- 6
  masterBlock[2,3] <- 6
  masterBlock[2,4] <- 6
  masterBlock[3,1] <- 6
  masterBlock[3,2] <- 6
  masterBlock[3,4] <- 6
  masterBlock[4,1] <- 6
  masterBlock[4,2] <- 6
  masterBlock[4,3] <- 6
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 7, 6)
  idparslist[[3]] <- myQ
  myQ

NA,6,6,6,6,0,0,0,6,0,0,0,6,0,0,0
6,NA,6,6,0,6,0,0,0,6,0,0,0,6,0,0
6,6,NA,6,0,0,6,0,0,0,6,0,0,0,6,0
6,6,6,NA,0,0,0,6,0,0,0,6,0,0,0,6
6,0,0,0,NA,6,6,6,6,0,0,0,6,0,0,0
0,6,0,0,6,NA,6,6,0,6,0,0,0,6,0,0
0,0,6,0,6,6,NA,6,0,0,6,0,0,0,6,0
0,0,0,6,6,6,6,NA,0,0,0,6,0,0,0,6
6,0,0,0,6,0,0,0,NA,6,6,6,6,0,0,0
0,6,0,0,0,6,0,0,6,NA,6,6,0,6,0,0
0,0,6,0,0,0,6,0,6,6,NA,6,0,0,6,0


In [30]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,4), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:6)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

ETD_sp<-secsse_ml(phy2.0,traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2287.346 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2287.34637051797 initial 
2 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00875303062128823 0.256218905472637 -2286.19680347946 expand 
3 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 expand 
4 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 reflect 
5 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2285.34831611282 reflect 
6 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.029242752208049

56 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 contract inside 
57 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 contract inside 
58 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 contract inside 
59 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 contract inside 
60 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 contract inside 
61 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.0125405749485243 0.000494346103195127 -1905.16228981007 reflect 
62 0.0289573580637975 0.0350452912783474 0.0257092102707428 0.0524259077138578 0.012

111 0.0269253720306704 0.0341749170326173 0.0307008708122104 0.0530338566736525 0.0116184014303345 0.000468605495208953 -1903.27950581403 reflect 
112 0.0269253720306704 0.0341749170326173 0.0307008708122104 0.0530338566736525 0.0116184014303345 0.000468605495208953 -1903.27950581403 reflect 
113 0.0269253720306704 0.0341749170326173 0.0307008708122104 0.0530338566736525 0.0116184014303345 0.000468605495208953 -1903.27950581403 reflect 
114 0.0273512281418631 0.0342911790525933 0.0299579905383344 0.0528729782612345 0.0117631571195921 0.000529942608425124 -1903.24224998767 reflect 
115 0.0266712457216439 0.0338705356617322 0.0319526887172434 0.0530705011588911 0.0114480092575238 0.000486632879074894 -1902.90980355482 expand 
116 0.0266712457216439 0.0338705356617322 0.0319526887172434 0.0530705011588911 0.0114480092575238 0.000486632879074894 -1902.90980355482 reflect 
117 0.0266712457216439 0.0338705356617322 0.0319526887172434 0.0530705011588911 0.0114480092575238 0.000486632879074894

167 0.0366686515251923 0.02012532673487 0.0399343076069762 0.0478051595639411 0.0196515245905383 0.000458740915692879 -1880.51638762752 contract inside 
168 0.0368795050625103 0.0200483005684409 0.0396660291038529 0.0477310643434068 0.0197969538684121 0.000445671034024701 -1880.51563132918 contract inside 
169 0.0368644950113054 0.0195603810436643 0.0406820272445544 0.0476654738381965 0.0198642764498884 0.000457850849855062 -1880.49932507519 contract inside 
170 0.0368644950113054 0.0195603810436643 0.0406820272445544 0.0476654738381965 0.0198642764498884 0.000457850849855062 -1880.49932507519 contract outside 
171 0.0368644950113054 0.0195603810436643 0.0406820272445544 0.0476654738381965 0.0198642764498884 0.000457850849855062 -1880.49932507519 contract inside 
172 0.037118129185743 0.019370276554175 0.0406275253006819 0.0475505063633844 0.0200330834522847 0.000454597512704682 -1880.48670410531 contract inside 
173 0.037118129185743 0.019370276554175 0.0406275253006819 0.047550506363

223 0.0383238550971005 0.0205784017306018 0.0404552304227484 0.0469304380256786 0.019034576338696 0.000478900346736755 -1880.11221388202 reflect 
224 0.0383238550971005 0.0205784017306018 0.0404552304227484 0.0469304380256786 0.019034576338696 0.000478900346736755 -1880.11221388202 contract inside 
225 0.0380578710134904 0.0200101472754482 0.0409180372067123 0.0470411189051513 0.0193662878618375 0.000466544194705358 -1880.10462854982 reflect 
226 0.0380578710134904 0.0200101472754482 0.0409180372067123 0.0470411189051513 0.0193662878618375 0.000466544194705358 -1880.10462854982 reflect 
227 0.0380578710134904 0.0200101472754482 0.0409180372067123 0.0470411189051513 0.0193662878618375 0.000466544194705358 -1880.10462854982 reflect 
228 0.0380578710134904 0.0200101472754482 0.0409180372067123 0.0470411189051513 0.0193662878618375 0.000466544194705358 -1880.10462854982 reflect 
229 0.0384501672952513 0.0205875911969923 0.0401480143069538 0.0468940803548581 0.019130255866247 0.000461687011

...

560 0.0364220251421347 0.0303826983086638 0.0442859969128794 0.0387393991232011 0.0204687800825419 0.000417401204874014 -1871.42135042637 reflect 
561 0.0364220251421347 0.0303826983086638 0.0442859969128794 0.0387393991232011 0.0204687800825419 0.000417401204874014 -1871.42135042637 reflect 
562 0.0364220251421347 0.0303826983086638 0.0442859969128794 0.0387393991232011 0.0204687800825419 0.000417401204874014 -1871.42135042637 contract inside 
563 0.0364220251421347 0.0303826983086638 0.0442859969128794 0.0387393991232011 0.0204687800825419 0.000417401204874014 -1871.42135042637 contract inside 
564 0.036500084600906 0.0301595159708834 0.044114976158007 0.0388523138847136 0.0205654167253044 0.000419592150365135 -1871.41728674872 expand 
565 0.036500084600906 0.0301595159708834 0.044114976158007 0.0388523138847136 0.0205654167253044 0.000419592150365135 -1871.41728674872 reflect 
566 0.0363333352010224 0.0305116914349906 0.0442278628336933 0.0386621909753637 0.0204268516516373 0.000418

617 0.0349910632591215 0.0269106015986296 0.0424319319692147 0.0362068245469213 0.0182337162726911 0.000428353340656719 -1870.76608548947 expand 
618 0.0340515092174282 0.0260701955714355 0.0405523470647852 0.0359250688373144 0.0168414112502372 0.000482271281059353 -1870.63325965476 expand 
619 0.0340515092174282 0.0260701955714355 0.0405523470647852 0.0359250688373144 0.0168414112502372 0.000482271281059353 -1870.63325965476 reflect 
620 0.0330610567244965 0.02776986925486 0.040647610497308 0.0358972609257718 0.0161335895916981 0.000444964249084705 -1870.5179182144 expand 
621 0.0329132412086054 0.0256606668199473 0.0392234353714211 0.036141849709003 0.0162218645112168 0.000421990852896028 -1870.40707908269 expand 
622 0.0324976212296614 0.0232047921225552 0.0392727380663488 0.034932645718964 0.0151234381569209 0.000422959653950718 -1870.23784313727 expand 
623 0.0324976212296614 0.0232047921225552 0.0392727380663488 0.034932645718964 0.0151234381569209 0.000422959653950718 -1870.2378

673 0.0307510610209492 0.0228134281011999 0.0385328446580956 0.0332926033918956 0.012568921715006 0.000413188413415741 -1870.03042170666 contract outside 
674 0.0308414572103263 0.0227464584027808 0.0384407201088137 0.0332771170162899 0.012592650082328 0.000412980523418321 -1870.03036312624 contract inside 
675 0.0308819990257758 0.0227995909112945 0.0385262696001546 0.033402028734259 0.0126120861173738 0.000416012798709672 -1870.02694873184 expand 
676 0.0308819990257758 0.0227995909112945 0.0385262696001546 0.033402028734259 0.0126120861173738 0.000416012798709672 -1870.02694873184 contract outside 
677 0.0308819990257758 0.0227995909112945 0.0385262696001546 0.033402028734259 0.0126120861173738 0.000416012798709672 -1870.02694873184 reflect 
678 0.0308819990257758 0.0227995909112945 0.0385262696001546 0.033402028734259 0.0126120861173738 0.000416012798709672 -1870.02694873184 reflect 
679 0.0308819990257758 0.0227995909112945 0.0385262696001546 0.033402028734259 0.0126120861173738 0

728 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 contract outside 
729 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 contract inside 
730 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 contract outside 
731 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 contract outside 
732 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 reflect 
733 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0125909756994739 0.000418303110650555 -1870.02630059199 reflect 
734 0.0308718718649774 0.0227075942219842 0.0385609539139311 0.0334238228380414 0.0

783 0.0308695981471528 0.0227037626438466 0.0385586268334835 0.0334224678621347 0.0125892017021677 0.000418382529674963 -1870.02629978539 contract inside 
784 0.0308695981471528 0.0227037626438466 0.0385586268334835 0.0334224678621347 0.0125892017021677 0.000418382529674963 -1870.02629978539 reflect 
785 0.0308695981471528 0.0227037626438466 0.0385586268334835 0.0334224678621347 0.0125892017021677 0.000418382529674963 -1870.02629978539 reflect 
786 0.0308695981471528 0.0227037626438466 0.0385586268334835 0.0334224678621347 0.0125892017021677 0.000418382529674963 -1870.02629978539 contract inside 
787 0.0308695981471528 0.0227037626438466 0.0385586268334835 0.0334224678621347 0.0125892017021677 0.000418382529674963 -1870.02629978539 contract inside 
788 0.030869586190852 0.0227037347581896 0.0385588048008788 0.0334224181773345 0.0125893175063383 0.000418388759336871 -1870.02629978448 reflect 
789 0.0308696004313952 0.0227037951865075 0.0385588616974322 0.0334225784168679 0.0125893439571

### Saving Data

In [31]:
### Saving Data

In [32]:
save(CR_sp, CTD2_sp, CTD3_sp, CTD4_sp, ETD_sp, file = "reprod_data.rds")

### Comparing models

In [33]:
### Comparing models

In [34]:
# Aicc function
aicc<-function(ll,k){round((2*(-round(ll,4))+2*k+(2*k*(k+1))/(Ntip(phy2.0)-k-1)),3)}

In [35]:
aicc_vec<-c(aicc(CR_sp$ML, 3), aicc(CTD2_sp$ML, 4), aicc(CTD3_sp$ML, 5), aicc(CTD4_sp$ML, 6), aicc(ETD_sp$ML, 6))

In [36]:
reprod_model_comp<-as.data.frame(cbind(c(CR_sp$ML, CTD2_sp$ML, CTD3_sp$ML, CTD4_sp$ML, ETD_sp$ML), c(3, 4, 5, 6, 6), aicc_vec, round(akaike.weights(aicc_vec)$deltaAIC, 3), round(akaike.weights(aicc_vec)$weights, 3)))

In [37]:
colnames(reprod_model_comp)<-c("log likelihood", "number of parameters", "AICc", "Delta AIC", "AIC weights")
rownames(reprod_model_comp)<-c("Constant rate model", "Concealt trait 2", "Conceal trait 3", "Conceal trait 4", "Focal trait")

### Saving Data

In [38]:
### Saving Data

In [40]:
write.table(reprod_model_comp, "Path/to/your/exit.file", sep ="\t")